## US Public Domain Map

A mapping project in which I use only public-domain data sources to create a map of the US.

By Kenneth Burchfiel

Code released under the MIT license

Maps (aside from the leaflet-specific elements, specifically the layer overlay, 'Leaflet' text on the bottom right, and zoom in/zoom out buttons) released into the public domain

In [1]:
import time
start_time = time.time()
import geopandas
import pandas as pd
import folium
import branca
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
# Source: https://selenium-python.readthedocs.io/getting-started.html
import PIL.Image
import IPython
from map_functions import create_map, create_map_screenshot, \
convert_png_to_smaller_jpg, add_alaska_and_hawaii


In [2]:
generate_water_features = False
generate_states = False
generate_roads = False
generate_places = False

In [3]:
if generate_water_features == True:
    # Note: this block took 11 minutes to execute on my laptop when 
    # generate_water_features was set to True.
    print("Reading file:")
    us_water_bodies = geopandas.read_file(r'C:\Users\kburc\Downloads\tlgdb_2021_a_us_areawater.gdb\tlgdb_2021_a_us_areawater.gdb')
    # File source: 
    # https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-geodatabase-file.2021.html
    # (Click on "Areal Hydrography National Geodatabase" to download the 
    # .zip file). Even though 'tlgdb_2021_a_us_areawater.gdb' is actually 
    # a folder, GeoPandas is still able to read it into a GeoDataFrame.
    
    # major_us_water_bodies = us_water_bodies.query("AWATER > 10000000").copy()
    print("Simplifying data:")
    us_water_bodies['geometry'] = us_water_bodies[
        'geometry'].simplify(tolerance = 0.005)
    print(pd.value_counts(us_water_bodies['MTFCC']))
    us_water_bodies_smaller = us_water_bodies.query("AWATER >= 1000000")
    # This statement creates a new DataFrame containing only features at least
    # 1 million square meters in size (equivalent to one square kilometer).
    # I was able to create a map using the entire dataset, but the resulting
    # .HTML file was 1.24 GB in size and wouldn't load on Firefox or Chrome.
    print(pd.value_counts(us_water_bodies_smaller['MTFCC']))
    print("Saving file:")
    us_water_bodies_smaller.to_file('us_water_bodies_simplified_smaller.geojson')

else:
    us_water_bodies_smaller = geopandas.read_file('us_water_bodies_simplified_smaller.geojson')


In [4]:
if generate_states == True:
    states = geopandas.read_file(r'C:\Users\kburc\Downloads\tl_2021_us_state\tl_2021_us_state.shp')
    # Source: https://www.census.gov/cgi-bin/geo/shapefiles/index.php
    states['geometry'] = states['geometry'].simplify(tolerance = 0.005)
    states.to_file('states_simplified.geojson')

else:
    states = geopandas.read_file('states_simplified.geojson')


In [5]:
if generate_roads == True:
    roads = geopandas.read_file(r'C:\Users\kburc\Downloads\tl_2021_us_primaryroads\tl_2021_us_primaryroads.shp')
    # Source: https://www.census.gov/cgi-bin/geo/shapefiles/index.php
    roads['geometry'] = roads['geometry'].simplify(tolerance = 0.005)
    roads.to_file('roads_simplified.geojson')

else:
    roads = geopandas.read_file('roads_simplified.geojson')


In [6]:
if generate_places == True:
    places = pd.read_csv(r'C:\Users\kburc\Downloads\2021_Gaz_place_national\2021_Gaz_place_national.txt', sep = "\t", header = 0)
    # Source: https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html
    # (Click on "Download the National Places Gazetteer Files" to access the 
    # file.)
    places.rename(columns={places.columns[11]:places.columns[11].strip()},inplace=True)
    # The INTPTLONG column has lots of extra spaces after it, hence the use of 
    # .strip here.
    places['GEOID'] = places['GEOID'].astype('str')
    places['GEOID'] = places['GEOID'].str.zfill(7)
    places.to_csv('places.csv', index = False)

    city_populations = pd.read_json(
        'https://api.census.gov/data/2020/dec/pl?get=NAME,P1_001N&for=place:*')
    # P1_001N is the population variable for the 2020 Decennial Census's 
    # redistricting data, as shown on:
    # https://api.census.gov/data/2020/dec/pl/variables.html
    # This API call derives from: 
    # https://api.census.gov/data/2020/dec/pl/examples.html
    city_populations.columns = city_populations.iloc[0] # Converts the first row into
    # the header column
    city_populations.drop(0, inplace = True)
    city_populations.rename(columns={"P1_001N":"population"}
    ,inplace=True)
    city_populations['population'] = pd.to_numeric(city_populations['population'])
    city_populations.sort_values('population', ascending = False, inplace = True)
    city_populations.reset_index(drop = True, inplace=True)
    # The following line calculates the GEOID value for each place based on its 
    # state and place codes. Since these are already in string format, there's
    # no need to convert them to strings in order to concatenate them.
    # These GEOID values will make an upcoming merge much easier, since both tables
    # have the same GEOID values for each city (whereas their 'NAME' values differ).
    # For more information about GEOID values, visit:
    # https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
    city_populations['GEOID'] = city_populations['state']+city_populations['place']
    city_populations.head(10)
    places.rename(columns = {'NAME':'place_name'}, inplace = True)
    cities_and_coords = city_populations.merge(places, on = 'GEOID', how = 'left')
    cities_and_coords.sort_values('population', ascending = False, inplace = True)
    cities_and_coords = cities_and_coords.query("population > 0")
    cities_and_coords.reset_index(drop = True, inplace = True)
    cities_and_coords.to_csv('cities_and_coords.csv', index = False) # Note that this file
    # is not a GeoDataFrame, but that's OK


else:
    cities_and_coords = pd.read_csv('cities_and_coords.csv')

In [7]:
lsad_codes = pd.read_csv('lsad_codes.csv') # I copied and pasted these codes from
# https://www.census.gov/library/reference/code-lists/legal-status-codes.html
# into a .csv file.
lsad_codes

,LSAD,LSAD Description,Associated Geographic Entity
0,0,NaN,"American Indian Area, Congressional District, ..."
1,3,City and Borough (suffix),County or Equivalent Feature
2,4,Borough (suffix),County or Equivalent Feature
3,5,Census Area (suffix),County or Equivalent Feature
4,6,Balance of County EC Place,Economic Census Place
...,...,...,...
131,UG,unified government (suffix),"Consolidated City, Economic Census Place, Inco..."
132,V1,Voting District (prefix),Voting District
133,V2,Voting District (suffix),Voting District
134,Z3,ZCTA3 (suffix),ZIP Code Tabulation Area (Three-Digit)


In [8]:
lsad_codes['New LSAD Description'] = lsad_codes['LSAD Description'].str.split("(").str[0].str.strip().str.lower()
lsad_codes['New LSAD Description'] = lsad_codes['New LSAD Description'].replace('cdp', 'CDP').replace('CCD', 'CDP')
# Splitting on " (" raised an error, so I used "(" and str.strip() instead.
lsad_codes.dropna(inplace=True)
lsad_codes[10:20]

,LSAD,LSAD Description,Associated Geographic Entity,New LSAD Description
11,20,barrio (suffix),County Subdivision,barrio
12,21,borough (suffix),"County Subdivision, Economic Census Place",borough
13,22,CCD (suffix),County Subdivision,ccd
14,23,census subarea (suffix),County Subdivision,census subarea
15,24,census subdistrict (suffix),County Subdivision,census subdistrict
16,25,city (suffix),"Consolidated City, County or Equivalent Featur...",city
17,26,county (suffix),"County Subdivision, Economic Census Place",county
18,27,district (suffix),"County Subdivision, Economic Census Place",district
19,28,District (prefix),"County Subdivision, Tribal Subdivision",district
20,29,precinct (suffix),County Subdivision,precinct


In [9]:
for description in lsad_codes['New LSAD Description']:
    cities_and_coords['place_name'] = cities_and_coords['place_name'].str.replace(description, '', case = True)
cities_and_coords['place_name'] = cities_and_coords['place_name'].str.replace('(balance)','', regex = False).str.replace('urban','').str.strip()
# regex = False needs to be added in here so that the code does not assume
# that the parentheses in the strings are part of a regular expression.
# See https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html

In [10]:
cities_and_coords['place_name'].head(50) 

0                        New York
1                     Los Angeles
2                         Chicago
3                         Houston
4                         Phoenix
5                    Philadelphia
6                     San Antonio
7                       San Diego
8                          Dallas
9                        San Jose
10                         Austin
11                   Jacksonville
12                     Fort Worth
13                       Columbus
14                   Indianapolis
15                      Charlotte
16                  San Francisco
17                        Seattle
18                         Denver
19                     Washington
20             Nashville-Davidson
21                  Oklahoma City
22                        El Paso
23                         Boston
24                       Portland
25                      Las Vegas
26                        Detroit
27                        Memphis
28                      Baltimore
29            

In [11]:
cities_and_coords.to_csv('revised_cities_and_coords.csv')

In [12]:
cities_and_coords

,NAME,population,state,place,GEOID,USPS,ANSICODE,place_name,LSAD,FUNCSTAT,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,"New York city, New York",8804190,36,51000,3651000,NY,2395220.0,New York,25,A,7.781782e+08,445408678.0,300.456,171.973,40.662712,-73.938677
1,"Los Angeles city, California",3898747,6,44000,644000,CA,2410877.0,Los Angeles,25,A,1.215977e+09,83031312.0,469.492,32.059,34.019394,-118.410825
2,"Chicago city, Illinois",2746388,17,14000,1714000,IL,428803.0,Chicago,25,A,5.898184e+08,17621589.0,227.730,6.804,41.837045,-87.684939
3,"Houston city, Texas",2304580,48,35000,4835000,TX,2410796.0,Houston,25,A,1.658719e+09,80896867.0,640.435,31.234,29.785743,-95.388806
4,"Phoenix city, Arizona",1608139,4,55000,455000,AZ,2411414.0,Phoenix,25,A,1.342304e+09,2633970.0,518.266,1.017,33.572154,-112.090132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31856,"Shannon Colony CDP, South Dakota",1,46,58332,4658332,SD,2813052.0,Shannon Colony,57,S,6.535240e+05,0.0,0.252,0.000,43.982612,-97.425074
31857,"Caribou CDP, California",1,6,11166,611166,CA,2407962.0,Caribou,57,S,4.197980e+05,0.0,0.162,0.000,40.073244,-121.165670
31858,"Ruso city, North Dakota",1,38,69140,3869140,ND,1036247.0,Ruso,25,A,6.436190e+05,0.0,0.249,0.000,47.837332,-100.934268
31859,"Hobart Bay CDP, Alaska",1,2,32550,232550,AK,2418615.0,Hobart Bay,57,S,3.093107e+08,24792456.0,119.426,9.572,57.417759,-133.306569


In [13]:
map_folder_path = r'C:\Users\kburc\D1V1\Documents\!Dell64docs\Programming\py\kjb3_programs\us_pub_domain_map'

In [14]:
create_map(gdf_water = us_water_bodies_smaller, gdf_states = states, 
gdf_places = cities_and_coords.iloc[0:100,:], gdf_roads = roads, save_path = 'pub_domain_features_map_smaller')
create_map_screenshot(absolute_path_to_map_folder = map_folder_path,
map_name = 'pub_domain_features_map_smaller.html', 
screenshot_save_path = 'screenshots')

Adding in bodies of water:
Adding in states:
Adding in roads:
Adding in cities:
Saving map:


In [15]:
add_alaska_and_hawaii(starting_map_name = 'pub_domain_features_map_smaller', absolute_path_to_map_folder = map_folder_path, tile_option = None, gdf_water = us_water_bodies_smaller, gdf_states = states, gdf_places = cities_and_coords.iloc[0:100,:], gdf_roads = roads)

Adding in bodies of water:
Adding in states:
Adding in roads:
Adding in cities:
Saving map:
Opened
Adding in bodies of water:
Adding in states:
Adding in roads:
Adding in cities:
Saving map:
Opened


In [16]:
create_map(gdf_water = us_water_bodies_smaller, gdf_states = states, 
gdf_places = cities_and_coords.iloc[0:100,:], gdf_roads = roads, save_path = 'pub_domain_features_map_smaller_no_cities', include_cities = False)
create_map_screenshot(absolute_path_to_map_folder = map_folder_path,
map_name = 'pub_domain_features_map_smaller_no_cities.html', 
screenshot_save_path = 'screenshots')
add_alaska_and_hawaii(starting_map_name = 'pub_domain_features_map_smaller_no_cities', absolute_path_to_map_folder = map_folder_path, tile_option = None, gdf_water = us_water_bodies_smaller, gdf_states = states, gdf_places = cities_and_coords.iloc[0:100,:], gdf_roads = roads, include_cities = False)

Adding in bodies of water:
Adding in states:
Adding in roads:
Saving map:
Adding in bodies of water:
Adding in states:
Adding in roads:
Saving map:
Opened
Adding in bodies of water:
Adding in states:
Adding in roads:
Saving map:
Opened


In [17]:
end_time = time.time()
run_time = end_time - start_time
run_minutes = run_time // 60
run_seconds = run_time % 60
print("Completed run at",time.ctime(end_time),"(local time)")
print("Total run time:",'{:.2f}'.format(run_time),
"second(s) ("+str(run_minutes),"minute(s) and",'{:.2f}'.format(run_seconds),
"second(s))") 
# Only meaningful when the program is run nonstop from start to finish

Completed run at Fri May 27 19:40:04 2022 (local time)
Total run time: 236.43 second(s) (3.0 minute(s) and 56.43 second(s))
